## Dataset preaparation

#### Import necessary packages

In [575]:
import numpy as np
import pandas as pd

#### Import Data form csv

In [576]:
# Define the paths to the files to import
text='/home/antonio/code/pollypineapple1/FED-Predictor/raw_data/Fed_Scrape-2015-2023.csv'
rates='/home/antonio/code/pollypineapple1/FED-Predictor/raw_data/US Fed Rate.csv'

In [577]:
# Import files
text_df=pd.read_csv(text)
rates_df=pd.read_csv(rates)

#### Check the data

In [578]:
text_df.head()

,Unnamed: 0,Date,Type,Text
0,0,20230412,0,The Federal Reserve on Wednesday released the ...
1,1,20230412,0,The minutes for each regularly scheduled meeti...
2,2,20230412,0,The minutes can be viewed on the Board's website.
3,3,20230412,0,"For media inquiries, e-mail [email protected] ..."
4,4,20230412,0,Minutes of the Federal Open Market Committee\r...


In [579]:
rates_df.head()

,Release Date,Time,Actual,Forecast,Previous
0,"Nov 01, 2023",13:00,5.50%,5.50%,5.50%
1,"Sep 20, 2023",13:00,5.50%,5.50%,5.50%
2,"Jul 26, 2023",13:00,5.50%,5.50%,5.25%
3,"Jun 14, 2023",13:00,5.25%,5.25%,5.25%
4,"May 03, 2023",13:00,5.25%,5.25%,5.00%


In [580]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  9974 non-null   int64 
 1   Date        9974 non-null   int64 
 2   Type        9974 non-null   int64 
 3   Text        9974 non-null   object
dtypes: int64(3), object(1)
memory usage: 311.8+ KB


In [581]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Release Date  201 non-null    object
 1   Time          201 non-null    object
 2   Actual        201 non-null    object
 3   Forecast      201 non-null    object
 4   Previous      201 non-null    object
dtypes: object(5)
memory usage: 8.0+ KB


#### Remove useless columns and format relevant ones

In [582]:
columns_to_keep = ['Release Date', 'Actual']
rates_df=rates_df[columns_to_keep]
rates_df=rates_df.rename(columns = {'Release Date' : 'date'})

In [583]:
rates_df['date'] = rates_df['date'].astype(str)

In [584]:
rates_df.head()

,date,Actual
0,"Nov 01, 2023",5.50%
1,"Sep 20, 2023",5.50%
2,"Jul 26, 2023",5.50%
3,"Jun 14, 2023",5.25%
4,"May 03, 2023",5.25%


Conversion to datetime

In [585]:
rates_df['date'] = pd.to_datetime(rates_df['date'], format = '%b %d, %Y')

In [586]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    201 non-null    datetime64[ns]
 1   Actual  201 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 3.3+ KB


In [587]:
rates_df=rates_df.rename(columns={'Actual': 'rate'})

In [588]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    201 non-null    datetime64[ns]
 1   rate    201 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 3.3+ KB


In [589]:
rates_df.head()

,date,rate
0,2023-11-01,5.50%
1,2023-09-20,5.50%
2,2023-07-26,5.50%
3,2023-06-14,5.25%
4,2023-05-03,5.25%


In [590]:
text_df['Date']=pd.to_datetime(text_df['Date'], format='%Y%m%d')
text_df=text_df.rename(columns={'Date': 'date'})

In [591]:
text_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [592]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    9974 non-null   datetime64[ns]
 1   Type    9974 non-null   int64         
 2   Text    9974 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 233.9+ KB


#### Merge the two dataframes based on Date

Sort by dates

In [593]:
sorted_rates=rates_df.sort_values(by='date')
sorted_text=text_df.sort_values(by='date')

In [594]:
sorted_rates.head()

,date,rate
200,1990-07-13,8.00%
199,1990-10-29,7.75%
198,1990-11-14,7.50%
197,1990-12-07,7.25%
196,1990-12-19,7.00%


In [595]:
sorted_text.head()

,date,Type,Text
9973,2015-01-07,0,"\r\n For media inquiries, call 202-452-2..."
9969,2015-01-07,0,\r\n The Federal Reserve Board and the F...
9972,2015-01-07,0,\nMinutes of the Federal Open Market Committee...
9971,2015-01-07,0,\r\n FOMC minutes can be viewed on the B...
9970,2015-01-07,0,\r\n The minutes for each regularly sche...


Merge based on date. >> Direction backward << to ensure rates decisions are paired with previous text

In [596]:
df_merged = pd.merge_asof(sorted_text, sorted_rates, on='date')
df_merged.head()

,date,Type,Text,rate
0,2015-01-07,0,"\r\n For media inquiries, call 202-452-2...",0.25%
1,2015-01-07,0,\r\n The Federal Reserve Board and the F...,0.25%
2,2015-01-07,0,\nMinutes of the Federal Open Market Committee...,0.25%
3,2015-01-07,0,\r\n FOMC minutes can be viewed on the B...,0.25%
4,2015-01-07,0,\r\n The minutes for each regularly sche...,0.25%


In [597]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    9974 non-null   datetime64[ns]
 1   Type    9974 non-null   int64         
 2   Text    9974 non-null   object        
 3   rate    9974 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 311.8+ KB
